<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Get-Pretrained-Params" data-toc-modified-id="Get-Pretrained-Params-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Get Pretrained Params</a></span></li><li><span><a href="#Transfer-Weights" data-toc-modified-id="Transfer-Weights-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Transfer Weights</a></span></li><li><span><a href="#Prune-w/-SBP" data-toc-modified-id="Prune-w/-SBP-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Prune w/ SBP</a></span></li><li><span><a href="#Resume-Training" data-toc-modified-id="Resume-Training-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Resume Training</a></span></li><li><span><a href="#Get-Mask-&amp;-Prune-Network" data-toc-modified-id="Get-Mask-&amp;-Prune-Network-0.5"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Get Mask &amp; Prune Network</a></span></li><li><span><a href="#PTFLOPS" data-toc-modified-id="PTFLOPS-0.6"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>PTFLOPS</a></span></li><li><span><a href="#Flop-weighted-importance" data-toc-modified-id="Flop-weighted-importance-0.7"><span class="toc-item-num">0.7&nbsp;&nbsp;</span>Flop weighted importance</a></span></li><li><span><a href="#Layer-index" data-toc-modified-id="Layer-index-0.8"><span class="toc-item-num">0.8&nbsp;&nbsp;</span>Layer index</a></span></li><li><span><a href="#Correlated-Net" data-toc-modified-id="Correlated-Net-0.9"><span class="toc-item-num">0.9&nbsp;&nbsp;</span>Correlated Net</a></span></li><li><span><a href="#Decorrelated-net" data-toc-modified-id="Decorrelated-net-0.10"><span class="toc-item-num">0.10&nbsp;&nbsp;</span>Decorrelated net</a></span></li></ul></li><li><span><a href="#Subplots" data-toc-modified-id="Subplots-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Subplots</a></span><ul class="toc-item"><li><span><a href="#Net-Slim-Train" data-toc-modified-id="Net-Slim-Train-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Net-Slim Train</a></span></li><li><span><a href="#L2-based-pruning-Train" data-toc-modified-id="L2-based-pruning-Train-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>L2 based pruning Train</a></span><ul class="toc-item"><li><span><a href="#Importance-plots-Netslim-Train" data-toc-modified-id="Importance-plots-Netslim-Train-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Importance plots Netslim Train</a></span></li><li><span><a href="#Importance-plots-L2-train" data-toc-modified-id="Importance-plots-L2-train-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Importance plots L2 train</a></span></li></ul></li></ul></li></ul></div>

In [1]:
# -*- coding: utf-8 -*-/
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm.autonotebook import tqdm

In [2]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
from utils import progress_bar

In [3]:
from SBP_alexnet import SBPConv_AlexNet
import SBP_utils_gpu as SBP_utils
import torch
import torch.nn as nn

In [4]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(45),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
     ])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
     ])

trainset = torchvision.datasets.CIFAR100(root='./../data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./../data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class AlexNet(nn.Module):

    def __init__(self, cfg, classes=100):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, cfg[0], kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(cfg[0]),
            
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(cfg[0], cfg[1], kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(cfg[1]),
            
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(cfg[1], cfg[2], kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(cfg[2]),
            
            nn.Conv2d(cfg[2], cfg[3], kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(cfg[3]),
            
            nn.Conv2d(cfg[3], cfg[4], kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(cfg[4]),
            
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(cfg[4] * 1 * 1, cfg[5]),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(cfg[5], cfg[6]),
            nn.ReLU(inplace=True),
            nn.Linear(cfg[6], classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
def net_test(epoch,net, criterion = nn.CrossEntropyLoss()):
    global best_base_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_base_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'best_acc': acc
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/alex_ckpt.pth')
        best_base_acc = acc

In [8]:
# Training
def SBP_net_train(epoch,net,optimizer,criterion = nn.CrossEntropyLoss(),lr_adjust=None, scheduler=None):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs,kl = net(inputs)
        
        #have to add the KL divergence while training for the SBP layers
        loss = criterion(outputs, targets) + kl
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    if lr_adjust:
        lr_adjust(optimizer,epoch)
    if scheduler:
        scheduler.step()
    return 
def SBP_net_test(epoch,net,criterion= nn.CrossEntropyLoss()):
    global best_sbp_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item() 
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_sbp_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'best_acc': acc
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/sbp_ckpt.pth')
        best_spb_acc = acc
         
    net.get_sparsity()
    return acc



In [9]:
from __future__ import print_function, absolute_import

__all__ = ['accuracy']

def kaccuracy(output, target, topk=(5,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def top5cal(net):
    net.eval()
    correct = 0
    total = 0
    top1 = 0
    top5 = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            acc1, acc5 = kaccuracy(outputs, targets, topk=(1, 5))
            top1 += (acc1.item()*inputs.shape[0])
            top5 += (acc5.item()*inputs.shape[0])
    top1 /= 10000
    top5 /= 10000
    
    print("top5", top5)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Get Pretrained Params

In [11]:
cfg = [64, 192, 384, 256, 256, 4096, 4096]
best_sbp_acc = best_base_acc = 0

In [12]:
best_net = AlexNet(cfg).to(device)

In [13]:
net_dict = torch.load('./pretrained_alex.pth')
best_net.load_state_dict(net_dict['net'])
best_acc = net_dict['best_acc']

In [14]:
best_net

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Conv2d(2

In [15]:
## check pretrained accuracy 51%
criterion = nn.CrossEntropyLoss()

net_test(1,best_net)

 [========================>]  Step: 7ms | Tot: 794ms | Loss: 1.950 | Acc: 50.960% (5096/1000 79/79 43/79 ........]  Step: 12ms | Tot: 459ms | Loss: 1.948 | Acc: 50.538% (2911/576 45/7 56/79 
Saving..


In [16]:
print('best_base_acc: ',best_base_acc)

best_base_acc:  50.96


### Transfer Weights 

In [17]:
best_sbp_acc = best_base_acc = 0 #reset best accuracy to save after running SBP

In [18]:
kl_weights = cfg
all_ones = [1] * 10
sbp_net = SBPConv_AlexNet(cfg,kl_weights=all_ones).to(device)

In [19]:
best_net

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Conv2d(2

In [27]:
#block 1

def transfer_weights(sbp_net):
    sbp_net.block1.conv1.weight = best_net.features[0].weight
    sbp_net.block1.conv1.bias = best_net.features[0].bias
    sbp_net.block1.bn1.weight = best_net.features[2].weight
    sbp_net.block1.bn1.bias = best_net.features[2].bias

    #block 2
    sbp_net.block2.conv1.weight = best_net.features[4].weight
    sbp_net.block2.conv1.bias = best_net.features[4].bias
    sbp_net.block2.bn1.weight = best_net.features[6].weight
    sbp_net.block2.bn1.bias = best_net.features[6].bias

    #block 3
    sbp_net.block3.conv1.weight = best_net.features[8].weight
    sbp_net.block3.conv1.bias = best_net.features[8].bias
    sbp_net.block3.bn1.weight = best_net.features[10].weight
    sbp_net.block3.bn1.bias = best_net.features[10].bias

    #block 4
    sbp_net.block4.conv1.weight = best_net.features[11].weight
    sbp_net.block4.conv1.bias = best_net.features[11].bias
    sbp_net.block4.bn1.weight = best_net.features[13].weight
    sbp_net.block4.bn1.bias = best_net.features[13].bias


    #block 5
    sbp_net.block5.conv1.weight = best_net.features[14].weight
    sbp_net.block5.conv1.bias = best_net.features[14].bias
    sbp_net.block5.bn1.weight = best_net.features[16].weight
    sbp_net.block5.bn1.bias = best_net.features[16].bias

    #note! Im not using SBP layers in the classifier right now. 
    #Otherwise, would need to transfer code here as well!
    sbp_net.lsbp1.weight = best_net.classifier[1].weight
    sbp_net.lsbp1.bias = best_net.classifier[1].bias

    sbp_net.lsbp2.weight = best_net.classifier[4].weight
    sbp_net.lsbp2.bias = best_net.classifier[4].bias
    
    return sbp_net

### Test Weights After Transferring

In [22]:
#its not that terrifying that the accuracy is trash 
criterion = nn.CrossEntropyLoss()
SBP_net_test(1,sbp_net,criterion=criterion)

 [========================>]  Step: 45ms | Tot: 3s815ms | Loss: 4.613 | Acc: 1.000% (100/1000 79/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(246.9508, device='cuda:0'), tensor(246.9508, device='cuda:0'), tensor(246.9508, device='cuda:0'), tensor(246.9508, device='cuda:0'), tensor(246.9508, device='cuda:0'), tensor(246.9508, device='cuda:0'), tensor(246.9508, device='cuda:0')]


1.0

### Prune w/ SBP 

In [24]:
#First attempt w/ lr. This is no longer being used. Here for completeness.

sbp_learningrate = 1e-5
finetune_epoch = 300 ## that seems excessive

kl_weights = cfg
all_ones = [1] * 10
sbp_net = SBPConv_AlexNet(cfg,kl_weights=cfg).to(device)

optimizer = optim.Adam(sbp_net.parameters(), lr=sbp_learningrate, betas=[0.95,0.999])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size= 250,gamma=0.1)

In [25]:
for epoch in range(0,300):
    SBP_net_train(epoch,sbp_net,optimizer=optimizer,criterion=nn.CrossEntropyLoss())
    SBP_net_test(epoch,sbp_net)


Epoch: 0
 [========================>]  Step: 69ms | Tot: 24s49ms | Loss: 66854.630 | Acc: 18.832% (9416/5000 391/391  1 69/391 .......]  Step: 68ms | Tot: 4s938ms | Loss: 67715.683 | Acc: 4.124% (417/1011 79/391 ....]  Step: 64ms | Tot: 5s71ms | Loss: 67710.069 | Acc: 4.167% (432/1036 81/391 =====>...................]  Step: 58ms | Tot: 5s732ms | Loss: 67682.008 | Acc: 4.851% (565/1164 91/391 ===============>.........]  Step: 61ms | Tot: 14s609ms | Loss: 67272.197 | Acc: 12.726% (3877/3046 238/391 379/391 380/39 389/391 
 [========================>]  Step: 43ms | Tot: 3s523ms | Loss: 3.620 | Acc: 37.410% (3741/1000 79/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(207.3631, device='cuda:0'), tensor(207.3631, device='cuda:0'), tensor(207.3631, device='cuda:0'), tensor(207.3631, device='cuda:0'), tensor(207.3631, device='cuda:0'), tensor(207.9739, device='cuda:0'), tensor(207.9592, device='cuda:0')]

Epoch: 1
 [========================>]  Step: 70ms | Tot: 24s731ms | L

 [========================>]  Step: 60ms | Tot: 23s756ms | Loss: 61660.916 | Acc: 57.142% (28571/5000 391/391  Step: 66ms | Tot: 1s271ms | Loss: 61777.260 | Acc: 55.841% (1501/268 21/39 43/39 44/391 45/391 ==>......................]  Step: 67ms | Tot: 2s873ms | Loss: 61768.272 | Acc: 55.859% (3289/588 46/391 51/391 58/39 61/391 
 [========================>]  Step: 43ms | Tot: 3s511ms | Loss: 1.946 | Acc: 49.960% (4996/1000 79/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(149.0000, device='cuda:0'), tensor(149.0020, device='cuda:0'), tensor(148.9927, device='cuda:0'), tensor(149.0003, device='cuda:0'), tensor(149.0062, device='cuda:0'), tensor(148.6904, device='cuda:0'), tensor(148.6919, device='cuda:0')]

Epoch: 6
 [========================>]  Step: 61ms | Tot: 25s257ms | Loss: 61476.320 | Acc: 57.572% (28786/5000 391/391 : 68ms | Tot: 265ms | Loss: 61554.082 | Acc: 54.688% (350/64 5/391  6/39 9/391 ....]  Step: 65ms | Tot: 591ms | Loss: 61552.936 | Acc: 55.781% (714

 [========================>]  Step: 67ms | Tot: 25s59ms | Loss: 60660.040 | Acc: 60.402% (30201/50000 390/391 91 50/39 52/391 ...............]  Step: 63ms | Tot: 3s309ms | Loss: 60676.218 | Acc: 59.259% (4096/691 54/391 ...............]  Step: 65ms | Tot: 3s964ms | Loss: 60675.729 | Acc: 59.607% (4883/819 64/39 119/391 .........]  Step: 68ms | Tot: 8s470ms | Loss: 60672.212 | Acc: 60.179% (10553/1753 137/391 ........]  Step: 63ms | Tot: 8s533ms | Loss: 60672.165 | Acc: 60.190% (10632/1766 138/391 154/391 >...............]  Step: 64ms | Tot: 9s743ms | Loss: 60671.254 | Acc: 60.047% (12067/20 157/391 ....]  Step: 67ms | Tot: 10s703ms | Loss: 60670.536 | Acc: 60.043% (13219/2201 172/391 174/39 180/391 =====>.............]  Step: 68ms | Tot: 11s722ms | Loss: 60669.814 | Acc: 60.119% (14390/2393 187/391 .....]  Step: 67ms | Tot: 11s789ms | Loss: 60669.766 | Acc: 60.140% (14472/2406 188/39 189/39 191/39 194/39 203/391 ....]  Step: 64ms | Tot: 12s845ms | Loss: 60669.004 | Acc: 59.995% (15666/

 [========================>]  Step: 60ms | Tot: 25s254ms | Loss: 60327.485 | Acc: 61.442% (30721/5000 391/391 65/391 .........]  Step: 66ms | Tot: 4s331ms | Loss: 60342.637 | Acc: 60.485% (5342/883 69/39 73/391 .................]  Step: 69ms | Tot: 4s663ms | Loss: 60342.397 | Acc: 60.631% (5743/947 74/391 81/39 82/39 91/39 95/391 98/391 .]  Step: 64ms | Tot: 6s885ms | Loss: 60340.782 | Acc: 60.894% (8418/1382 108/391 112/39 116/391 ............]  Step: 65ms | Tot: 8s438ms | Loss: 60339.653 | Acc: 60.991% (10305/1689 132/39 145/391 ..........]  Step: 69ms | Tot: 10s354ms | Loss: 60338.244 | Acc: 60.986% (12646/2073 162/391 ]  Step: 69ms | Tot: 10s879ms | Loss: 60337.865 | Acc: 61.167% (13310/2176 170/39 174/391 176/391 ===========>.............]  Step: 70ms | Tot: 12s59ms | Loss: 60337.021 | Acc: 61.041% (14689/2406 188/391 .......]  Step: 71ms | Tot: 12s452ms | Loss: 60336.741 | Acc: 60.978% (15142/2483 194/391 ............]  Step: 71ms | Tot: 12s845ms | Loss: 60336.459 | Acc: 60.973% 

 [========================>]  Step: 49ms | Tot: 19s388ms | Loss: 60070.976 | Acc: 62.093% (30997/4992 390/391 /39 11/391 .......]  Step: 48ms | Tot: 848ms | Loss: 60088.456 | Acc: 60.200% (1387/230 18/39 24/391 ...]  Step: 51ms | Tot: 1s192ms | Loss: 60088.108 | Acc: 60.688% (1942/320 25/391 28/391 .............]  Step: 53ms | Tot: 1s393ms | Loss: 60087.916 | Acc: 60.749% (2255/371 29/39 32/391 36/391 ]  Step: 53ms | Tot: 1s788ms | Loss: 60087.558 | Acc: 60.790% (2879/473 37/391 .................]  Step: 47ms | Tot: 1s935ms | Loss: 60087.413 | Acc: 60.996% (3123/512 40/39 41/391 ..]  Step: 51ms | Tot: 2s231ms | Loss: 60087.126 | Acc: 61.294% (3609/588 46/391 .............]  Step: 49ms | Tot: 2s329ms | Loss: 60087.023 | Acc: 61.540% (3781/614 48/391 ..............]  Step: 50ms | Tot: 2s432ms | Loss: 60086.930 | Acc: 61.594% (3942/640 50/391 ===>.....................]  Step: 45ms | Tot: 2s582ms | Loss: 60086.787 | Acc: 61.557% (4176/678 53/39 54/391 55/391 60/39 76/391 .................]

 [========================>]  Step: 43ms | Tot: 3s749ms | Loss: 2.026 | Acc: 50.530% (5053/1000 79/79 .......]  Step: 45ms | Tot: 2s68ms | Loss: 2.026 | Acc: 50.260% (2895/576 45/79 .....]  Step: 45ms | Tot: 2s269ms | Loss: 2.030 | Acc: 50.223% (3150/627 49/79 ==========>.........]  Step: 54ms | Tot: 2s323ms | Loss: 2.025 | Acc: 50.328% (3221/640 50/79 =============>.........]  Step: 45ms | Tot: 2s369ms | Loss: 2.029 | Acc: 50.398% (3290/652 51/79 ======>........]  Step: 45ms | Tot: 2s469ms | Loss: 2.024 | Acc: 50.516% (3427/678 53/79 66/79 =================>...]  Step: 55ms | Tot: 3s326ms | Loss: 2.025 | Acc: 50.513% (4526/896 70/7 75/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(126.3112, device='cuda:0'), tensor(126.0520, device='cuda:0'), tensor(126.2500, device='cuda:0'), tensor(126.3692, device='cuda:0'), tensor(126.2307, device='cuda:0'), tensor(126.3785, device='cuda:0'), tensor(126.3878, device='cuda:0')]

Epoch: 40
 [========================>]  Step: 67ms |

 [========================>]  Step: 59ms | Tot: 24s177ms | Loss: 59595.121 | Acc: 63.956% (31978/5000 391/391 2/391 ........]  Step: 62ms | Tot: 9s772ms | Loss: 59605.985 | Acc: 63.576% (12939/2035 159/39 185/391 212/391 =======>......]  Step: 70ms | Tot: 18s54ms | Loss: 59599.755 | Acc: 63.907% (23886/3737 292/391   Step: 69ms | Tot: 18s316ms | Loss: 59599.566 | Acc: 63.975% (24239/3788 296/391 
 [========================>]  Step: 60ms | Tot: 3s474ms | Loss: 2.047 | Acc: 50.610% (5061/1000 79/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(121.0106, device='cuda:0'), tensor(120.8518, device='cuda:0'), tensor(120.8799, device='cuda:0'), tensor(120.8073, device='cuda:0'), tensor(120.8225, device='cuda:0'), tensor(120.8744, device='cuda:0'), tensor(120.8779, device='cuda:0')]

Epoch: 51
 [========================>]  Step: 62ms | Tot: 24s165ms | Loss: 59558.532 | Acc: 63.850% (31925/5000 391/391 ................]  Step: 70ms | Tot: 1s511ms | Loss: 59575.689 | Acc: 63.094%

KeyboardInterrupt: 

In [42]:
equal_optimizer.param_groups[0]['lr']

1e-05

## Train Equal KL Weights

In [22]:
#This is similar to the original paper's hyper-parameters for VGG. Also fails. 


best_sbp_acc = best_base_acc = 0 #reset best accuracy to save after running SBP

equal_weights = [1,1,1,1,1,1,1]
lr_x = [] #learning rate decay


def learning_rate_calc(optimizer, epoch):
    if epoch < 250:
        return 1e-5
    else: 
        return 1e-5 * (300-epoch)/(300-250)

sbp_learningrate = 1e-5
finetune_epoch = 300 ## that seems excessive
equal_optimizer = optim.Adam(sbp_equal_net.parameters(),lr=sbp_learningrate, betas=[0.95,0.999])
#equal_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_x)

for epoch in range(0,300):
    SBP_net_train(epoch,sbp_equal_net,optimizer=equal_optimizer,criterion=nn.CrossEntropyLoss(),lr_adjust=learning_rate_calc)
    SBP_net_test(epoch,sbp_equal_net)


Epoch: 0
 [========================>]  Step: 71ms | Tot: 22s651ms | Loss: 54.625 | Acc: 2.440% (1220/5000 391/391 43/391 .........]  Step: 67ms | Tot: 3s427ms | Loss: 55.425 | Acc: 1.353% (97/716 56/391 ]  Step: 59ms | Tot: 3s872ms | Loss: 55.405 | Acc: 1.438% (116/806 63/391 ............]  Step: 62ms | Tot: 8s709ms | Loss: 55.157 | Acc: 1.539% (329/2137 167/39 332/391 349/391 
 [========================>]  Step: 43ms | Tot: 3s590ms | Loss: 4.332 | Acc: 6.030% (603/1000 79/79 9 ====>..................]  Step: 46ms | Tot: 992ms | Loss: 4.325 | Acc: 6.001% (169/281 22/7 23/79 24/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(207.3795, device='cuda:0'), tensor(207.5979, device='cuda:0'), tensor(207.8234, device='cuda:0'), tensor(207.5694, device='cuda:0'), tensor(207.5939, device='cuda:0'), tensor(208.0278, device='cuda:0'), tensor(208.0318, device='cuda:0')]

Epoch: 1
 [========================>]  Step: 54ms | Tot: 24s735ms | Loss: 52.857 | Acc: 5.188% (2594/5000 391/3

 [========================>]  Step: 54ms | Tot: 24s308ms | Loss: 49.488 | Acc: 14.510% (7255/5000 391/391 1 ....................]  Step: 65ms | Tot: 3s77ms | Loss: 49.539 | Acc: 13.900% (943/678 53/391 54/391 63/391 ..........]  Step: 66ms | Tot: 3s868ms | Loss: 49.535 | Acc: 13.990% (1164/832 65/391 68/39 76/391 ..................]  Step: 66ms | Tot: 4s798ms | Loss: 49.533 | Acc: 13.944% (1410/10 79/391  81/391 ..........]  Step: 66ms | Tot: 5s445ms | Loss: 49.526 | Acc: 14.098% (1606/1139 89/39 92/39 104/391 >.................]  Step: 65ms | Tot: 7s82ms | Loss: 49.524 | Acc: 14.179% (2069/1459 114/391 .......]  Step: 67ms | Tot: 7s149ms | Loss: 49.524 | Acc: 14.219% (2093/1472 115/391 ===>.................]  Step: 67ms | Tot: 7s280ms | Loss: 49.522 | Acc: 14.276% (2138/1497 117/39 121/391 122/391 ==>.................]  Step: 69ms | Tot: 7s806ms | Loss: 49.524 | Acc: 14.294% (2287/16 125/391 ========>................]  Step: 61ms | Tot: 8s8ms | Loss: 49.527 | Acc: 14.258% (2336/1638 1

 [========================>]  Step: 54ms | Tot: 22s745ms | Loss: 48.840 | Acc: 19.408% (9704/5000 391/391 91 ]  Step: 61ms | Tot: 3s154ms | Loss: 48.873 | Acc: 19.455% (1270/652 51/39 52/391 .................]  Step: 63ms | Tot: 3s425ms | Loss: 48.872 | Acc: 19.517% (1374/704 55/391 .............]  Step: 67ms | Tot: 4s989ms | Loss: 48.867 | Acc: 19.219% (1968/1024 80/391 104/39 163/39 166/391 ======>.............]  Step: 48ms | Tot: 10s897ms | Loss: 48.855 | Acc: 19.212% (4328/2252 176/391 .......]  Step: 50ms | Tot: 11s254ms | Loss: 48.856 | Acc: 19.215% (4501/2342 183/39 185/391 ====>............]  Step: 49ms | Tot: 11s783ms | Loss: 48.859 | Acc: 19.167% (4735/2470 193/391 ===>..........]  Step: 52ms | Tot: 13s810ms | Loss: 48.849 | Acc: 19.312% (5735/2969 232/391 259/391   Step: 46ms | Tot: 15s687ms | Loss: 48.851 | Acc: 19.257% (6606/3430 268/391 ==>......]  Step: 51ms | Tot: 17s251ms | Loss: 48.845 | Acc: 19.347% (7355/3801 297/391 ====>....]  Step: 52ms | Tot: 18s203ms | Loss: 48

 [========================>]  Step: 57ms | Tot: 24s583ms | Loss: 48.239 | Acc: 23.342% (11671/5000 391/391 1 29/391 ..........]  Step: 65ms | Tot: 2s419ms | Loss: 48.269 | Acc: 22.871% (1171/512 40/391 55/391 56/391 .................]  Step: 61ms | Tot: 3s953ms | Loss: 48.260 | Acc: 22.986% (1883/819 64/391 ...]  Step: 68ms | Tot: 4s650ms | Loss: 48.250 | Acc: 23.115% (2219/960 75/391 109/391 .........]  Step: 69ms | Tot: 7s459ms | Loss: 48.246 | Acc: 23.190% (3562/1536 120/391 123/391 ..............]  Step: 69ms | Tot: 7s724ms | Loss: 48.248 | Acc: 23.154% (3675/1587 124/391 134/39 142/391 =====>...............]  Step: 69ms | Tot: 9s215ms | Loss: 48.246 | Acc: 23.140% (4354/1881 147/39 149/391 =========>...............]  Step: 64ms | Tot: 9s479ms | Loss: 48.248 | Acc: 23.075% (4460/1932 151/39 152/39 158/391 159/391 >..............]  Step: 64ms | Tot: 10s327ms | Loss: 48.244 | Acc: 23.104% (4850/2099 164/39 176/391 177/391 181/391 ........]  Step: 66ms | Tot: 11s749ms | Loss: 48.250 |

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

 [========================>]  Step: 50ms | Tot: 3s575ms | Loss: 2.250 | Acc: 42.490% (4249/1000 79/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(50.6206, device='cuda:0'), tensor(50.6058, device='cuda:0'), tensor(50.6519, device='cuda:0'), tensor(50.6182, device='cuda:0'), tensor(50.6309, device='cuda:0'), tensor(50.5682, device='cuda:0'), tensor(50.5898, device='cuda:0')]

Epoch: 275
 [========================>]  Step: 48ms | Tot: 22s934ms | Loss: 40.460 | Acc: 46.046% (23023/5000 391/391 ...............]  Step: 68ms | Tot: 2s279ms | Loss: 40.485 | Acc: 45.979% (2413/524 41/391 87/391 88/391 >..................]  Step: 67ms | Tot: 5s730ms | Loss: 40.445 | Acc: 46.612% (5668/1216 95/39 113/391  255/391 
 [========================>]  Step: 44ms | Tot: 3s573ms | Loss: 2.252 | Acc: 42.470% (4247/1000 79/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(50.4513, device='cuda:0'), tensor(50.4386, device='cuda:0'), tensor(50.4510, device='cuda:0'), tensor(50

 [========================>]  Step: 61ms | Tot: 24s372ms | Loss: 40.227 | Acc: 46.300% (23150/5000 391/391 ...............]  Step: 66ms | Tot: 8s909ms | Loss: 40.224 | Acc: 46.421% (8794/1894 148/39 164/391 =====>..............]  Step: 69ms | Tot: 9s986ms | Loss: 40.222 | Acc: 46.482% (9817/2112 165/391 =======>..............]  Step: 68ms | Tot: 10s55ms | Loss: 40.222 | Acc: 46.494% (9879/2124 166/39 167/391 171/391 ======>.............]  Step: 65ms | Tot: 10s580ms | Loss: 40.224 | Acc: 46.453% (10346/2227 174/39 177/391 =>.............]  Step: 68ms | Tot: 10s852ms | Loss: 40.225 | Acc: 46.414% (10575/2278 178/391 ........]  Step: 65ms | Tot: 11s751ms | Loss: 40.232 | Acc: 46.370% (11396/2457 192/39 194/391 ........]  Step: 68ms | Tot: 11s952ms | Loss: 40.232 | Acc: 46.306% (11558/2496 195/391 =>............]  Step: 69ms | Tot: 12s21ms | Loss: 40.233 | Acc: 46.297% (11615/2508 196/391 ==>............]  Step: 62ms | Tot: 12s84ms | Loss: 40.232 | Acc: 46.292% (11673/25 197/391 198/39 207

 [========================>]  Step: 63ms | Tot: 24s877ms | Loss: 40.105 | Acc: 46.390% (23195/5000 391/391 ..........]  Step: 71ms | Tot: 450ms | Loss: 40.172 | Acc: 44.629% (457/102 8/391 9/391 ..................]  Step: 63ms | Tot: 712ms | Loss: 40.130 | Acc: 46.159% (709/153 12/391 16/39 19/391 ...........]  Step: 66ms | Tot: 1s241ms | Loss: 40.113 | Acc: 46.445% (1189/256 20/391 =>.......................]  Step: 66ms | Tot: 1s308ms | Loss: 40.112 | Acc: 46.354% (1246/268 21/39 44/391 ...............]  Step: 66ms | Tot: 2s818ms | Loss: 40.148 | Acc: 45.365% (2613/576 45/39 50/39 69/39 91/391 ...]  Step: 66ms | Tot: 6s181ms | Loss: 40.113 | Acc: 46.078% (5721/1241 97/391 .................]  Step: 65ms | Tot: 6s514ms | Loss: 40.114 | Acc: 46.040% (6011/1305 102/39 104/391 ..........]  Step: 66ms | Tot: 6s714ms | Loss: 40.112 | Acc: 46.257% (6217/1344 105/391 >..................]  Step: 68ms | Tot: 6s847ms | Loss: 40.111 | Acc: 46.298% (6341/13 107/391 ...............]  Step: 62ms | To

 [========================>]  Step: 63ms | Tot: 25s684ms | Loss: 39.976 | Acc: 46.842% (23421/5000 391/391 7/391 ....................]  Step: 65ms | Tot: 534ms | Loss: 40.066 | Acc: 45.747% (527/115 9/391 .....................]  Step: 69ms | Tot: 1s145ms | Loss: 40.022 | Acc: 46.658% (1075/230 18/391 ................]  Step: 63ms | Tot: 1s208ms | Loss: 40.011 | Acc: 46.957% (1142/243 19/39 20/391 .....................]  Step: 69ms | Tot: 1s347ms | Loss: 40.013 | Acc: 46.763% (1257/268 21/391 ...............]  Step: 62ms | Tot: 1s410ms | Loss: 40.011 | Acc: 46.768% (1317/281 22/391 25/39 28/39 33/391 37/391 50/39 57/391 ............]  Step: 63ms | Tot: 4s189ms | Loss: 40.007 | Acc: 46.521% (3811/819 64/391 ..................]  Step: 62ms | Tot: 4s725ms | Loss: 39.998 | Acc: 46.658% (4300/921 72/39 82/391 ...............]  Step: 62ms | Tot: 5s673ms | Loss: 39.992 | Acc: 46.503% (5119/1100 86/391 100/391  106/391 ======>..................]  Step: 65ms | Tot: 7s153ms | Loss: 39.986 | Acc: 

 [========================>]  Step: 52ms | Tot: 23s753ms | Loss: 39.766 | Acc: 47.028% (23514/5000 391/391 .................]  Step: 66ms | Tot: 3s677ms | Loss: 39.778 | Acc: 46.593% (3638/780 61/391 63/391 .................]  Step: 59ms | Tot: 4s8ms | Loss: 39.773 | Acc: 46.650% (3941/844 66/39 217/391 =============>...........]  Step: 65ms | Tot: 13s203ms | Loss: 39.762 | Acc: 47.349% (13273/2803 219/391 230/391 245/391 ........]  Step: 68ms | Tot: 14s949ms | Loss: 39.762 | Acc: 47.283% (14949/3161 247/39 267/39 276/391 284/391 =============>......]  Step: 64ms | Tot: 17s734ms | Loss: 39.766 | Acc: 47.200% (17581/3724 291/39 303/391 >....]  Step: 69ms | Tot: 19s183ms | Loss: 39.769 | Acc: 47.074% (18920/4019 314/391 =============>....]  Step: 67ms | Tot: 19s454ms | Loss: 39.770 | Acc: 47.047% (19150/4070 318/391 ===============>....]  Step: 69ms | Tot: 19s970ms | Loss: 39.770 | Acc: 47.038% (19628/4172 326/391 ====================>....]  Step: 65ms | Tot: 20s36ms | Loss: 39.769 | Acc

## Try New Initialize Scheme

In [23]:
best_sbp_acc = best_base_acc = 0 #reset best accuracy to save after running SBP


In [ ]:
#This is the approach I've had the most success with in lowering the sparsity gradually from 0 to 1. 
#But it monotonically increases sparsity while destryoing the accuracy. 

sbp_learningrate = 2e-5
sbp_parameters = [
    {'params': sbp_net.block1.conv1.weight},
    {'params': sbp_net.block2.conv1.weight},
    {'params': sbp_net.block3.conv1.weight},
    {'params': sbp_net.block4.conv1.weight},
    {'params': sbp_net.block5.conv1.weight},
    
    {'params': sbp_net.block1.bn1.weight},
    {'params': sbp_net.block2.bn1.weight},
    {'params': sbp_net.block3.bn1.weight},
    {'params': sbp_net.block4.bn1.weight},
    {'params': sbp_net.block5.bn1.weight},
    
    {'params': sbp_net.block1.conv1.log_sigma, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block2.conv1.log_sigma, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block3.conv1.log_sigma, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block4.conv1.log_sigma, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block5.conv1.log_sigma, 'lr': 10*sbp_learningrate},

    {'params': sbp_net.block1.conv1.mu, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block2.conv1.mu, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block3.conv1.mu, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block4.conv1.mu, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.block5.conv1.mu, 'lr': 10*sbp_learningrate},

    {'params': sbp_net.lsbp1.mu, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.lsbp1.log_sigma, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.lsbp2.mu, 'lr': 10*sbp_learningrate},
    {'params': sbp_net.lsbp2.log_sigma, 'lr': 10*sbp_learningrate},
    
    {'params': sbp_net.last.weight},

    
 ]




finetune_epoch = 300 ## that seems excessive

sbp_net = SBPConv_AlexNet(cfg,kl_weights=all_ones).to(device)
sbp_net = transfer_weights(sbp_net)

sbp_optimizer = optim.Adam(sbp_parameters,lr=sbp_learningrate, betas=[0.95,0.999])
sbp_scheduler = optim.lr_scheduler.StepLR(sbp_optimizer,step_size=250, gamma=0.1)

#epoch 45 is close to the last epoch of death. 
for epoch in range(0,45):
    SBP_net_train(epoch,sbp_net,optimizer=sbp_optimizer,criterion=nn.CrossEntropyLoss(),scheduler=sbp_scheduler)
    SBP_net_test(epoch,sbp_net)


Epoch: 0
 [========================>]  Step: 64ms | Tot: 22s529ms | Loss: 50.622 | Acc: 13.110% (6555/5000 391/391  43/391 51/391 .............]  Step: 63ms | Tot: 6s197ms | Loss: 52.345 | Acc: 2.683% (340/1267 99/391 123/39 126/39 146/39 156/39 242/391 256/39 280/391 ========>....]  Step: 64ms | Tot: 17s882ms | Loss: 50.833 | Acc: 10.308% (4209/4083 319/39 321/391 .]  Step: 58ms | Tot: 18s569ms | Loss: 50.801 | Acc: 10.731% (4519/4211 329/391 ======================>..]  Step: 63ms | Tot: 19s659ms | Loss: 50.749 | Acc: 11.403% (5050/4428 346/391 ]  Step: 67ms | Tot: 20s608ms | Loss: 50.708 | Acc: 11.964% (5513/4608 360/391 ==>.]  Step: 65ms | Tot: 21s409ms | Loss: 50.671 | Acc: 12.477% (5957/4774 373/391 
 [========================>]  Step: 44ms | Tot: 3s649ms | Loss: 4.084 | Acc: 28.760% (2876/1000 79/79 /79 43/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(136.4969, device='cuda:0'), tensor(136.6378, device='cuda:0'), tensor(135.9762, device='cuda:0'), te

 [========================>]  Step: 68ms | Tot: 23s146ms | Loss: 43.822 | Acc: 54.662% (27331/5000 391/391  Step: 71ms | Tot: 1s62ms | Loss: 44.140 | Acc: 54.609% (1398/256 20/391 30/391 ==>......................]  Step: 48ms | Tot: 2s104ms | Loss: 44.139 | Acc: 53.652% (2747/512 40/391 .......]  Step: 42ms | Tot: 2s619ms | Loss: 44.131 | Acc: 53.799% (3512/652 51/391 >....................]  Step: 67ms | Tot: 3s564ms | Loss: 44.112 | Acc: 53.717% (4538/844 66/391 ..................]  Step: 67ms | Tot: 3s631ms | Loss: 44.114 | Acc: 53.638% (4600/857 67/391 ......]  Step: 64ms | Tot: 3s696ms | Loss: 44.113 | Acc: 53.653% (4670/870 68/391 .................]  Step: 66ms | Tot: 3s831ms | Loss: 44.110 | Acc: 53.683% (4810/896 70/391 71/391 ............]  Step: 66ms | Tot: 4s31ms | Loss: 44.108 | Acc: 53.660% (5014/934 73/391 82/391 95/391 ............]  Step: 64ms | Tot: 6s137ms | Loss: 44.066 | Acc: 53.936% (7249/1344 105/391 ..........]  Step: 62ms | Tot: 7s772ms | Loss: 44.045 | Acc: 54.0

 [========================>]  Step: 69ms | Tot: 23s791ms | Loss: 39.050 | Acc: 59.016% (29508/5000 391/391 ....]  Step: 66ms | Tot: 335ms | Loss: 39.377 | Acc: 58.203% (447/76 6/391 .............]  Step: 70ms | Tot: 535ms | Loss: 39.353 | Acc: 57.639% (664/115 9/391 36/391 .........]  Step: 51ms | Tot: 2s502ms | Loss: 39.345 | Acc: 56.942% (3207/563 44/391  45/39 71/39 76/391 >..................]  Step: 69ms | Tot: 5s753ms | Loss: 39.280 | Acc: 58.002% (7350/1267 99/391 ======>..................]  Step: 68ms | Tot: 6s501ms | Loss: 39.268 | Acc: 58.246% (8201/1408 110/391 ========>................]  Step: 68ms | Tot: 7s753ms | Loss: 39.255 | Acc: 58.267% (9621/1651 129/391 =>.............]  Step: 61ms | Tot: 10s504ms | Loss: 39.217 | Acc: 58.612% (13054/2227 174/39 206/39 245/391 ========>........]  Step: 57ms | Tot: 15s465ms | Loss: 39.151 | Acc: 59.058% (19352/3276 256/391 ==================>..]  Step: 69ms | Tot: 21s255ms | Loss: 39.077 | Acc: 59.065% (26688/4518 353/391 ============

 [========================>]  Step: 67ms | Tot: 23s738ms | Loss: 34.561 | Acc: 60.820% (30410/5000 391/391 ...................]  Step: 64ms | Tot: 1s666ms | Loss: 34.833 | Acc: 60.426% (2243/371 29/391 .................]  Step: 64ms | Tot: 3s524ms | Loss: 34.829 | Acc: 59.614% (4731/793 62/391 107/391 ...........]  Step: 65ms | Tot: 6s350ms | Loss: 34.777 | Acc: 60.026% (8298/1382 108/391 =>.................]  Step: 69ms | Tot: 6s549ms | Loss: 34.779 | Acc: 59.987% (8523/1420 111/391 ..........]  Step: 64ms | Tot: 6s614ms | Loss: 34.778 | Acc: 59.975% (8598/1433 112/391 142/391 160/39 212/391 =======>...........]  Step: 68ms | Tot: 12s557ms | Loss: 34.697 | Acc: 60.424% (16474/2726 213/39 215/391 ==============>..........]  Step: 61ms | Tot: 13s103ms | Loss: 34.690 | Acc: 60.453% (17101/2828 221/39 264/391 ==>.......]  Step: 65ms | Tot: 16s38ms | Loss: 34.651 | Acc: 60.615% (20871/3443 269/391 ===>.......]  Step: 66ms | Tot: 16s105ms | Loss: 34.650 | Acc: 60.631% (20954/3456 270/391 28

In [26]:
SBP_net_test(44,sbp_net)

 [========================>]  Step: 45ms | Tot: 3s769ms | Loss: 1.982 | Acc: 49.540% (4954/1000 79/79 1/79 
Saving..
Sparsity:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
SNRS:  [tensor(4.2766, device='cuda:0'), tensor(4.2796, device='cuda:0'), tensor(4.2981, device='cuda:0'), tensor(4.2887, device='cuda:0'), tensor(4.2776, device='cuda:0'), tensor(4.8969, device='cuda:0'), tensor(4.8912, device='cuda:0')]


49.54

In [29]:
with torch.cuda.device(-1):
    flops, params = get_model_complexity_info(best_net, (3, 32, 32), as_strings=True, print_per_layer_stat=False)
    print('{:<30}  {:<8}'.format('Computational complexity: ', flops))

Computational complexity:       0.04 GMac


In [28]:
from ptflops import get_model_complexity_info

#have to remove the sbp layers before calling this! B/c ptflops will not support the custom layers. 
with torch.cuda.device(-1):
    flops, params = get_model_complexity_info(sbp_net, (3, 32, 32), as_strings=True, print_per_layer_stat=False)
    print('{:<30}  {:<8}'.format('Computational complexity: ', flops))

Computational complexity:       0.0 GMac


## Try Warm-up learning Rate 

In [32]:
sbp_net.kl_weights = [1] * 10

In [37]:
best_sbp_acc = best_base_acc = 0 #reset best accuracy to save after running SBP

equal_weights = [1,1,1,1,1,1,1]
lr_x = [] #learning rate decay

def learning_rate_calc(optimizer, epoch):
    if epoch < 350:
        return 1e-5
    else: 
        return 1e-5 * (400-epoch)/(400-350)


sbp_learningrate = 1e-5
finetune_epoch = 300 ## that seems excessive
warmup_optimizer = optim.Adam(sbp_net.parameters(),lr=sbp_learningrate, betas=[0.95,0.999])
#equal_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_x)
warmup_scheduler = optim.lr_scheduler.MultiStepLR(warmup_optimizer,milestones=[10,20,50],gamma=0.1)

for epoch in range(0,100):
    SBP_net_train(epoch,sbp_net,optimizer=warmup_optimizer,criterion=nn.CrossEntropyLoss(),lr_adjust=None,scheduler=warmup_scheduler)
    SBP_net_test(epoch,sbp_net)


Epoch: 0
 [========================>]  Step: 62ms | Tot: 24s517ms | Loss: 12.203 | Acc: 63.534% (31767/5000 391/391 /391 33/39 53/39 85/39 159/39 179/391 ==>.............]  Step: 68ms | Tot: 11s266ms | Loss: 12.244 | Acc: 62.491% (14638/2342 183/391 ========>.............]  Step: 68ms | Tot: 11s400ms | Loss: 12.245 | Acc: 62.492% (14798/2368 185/391 ==>.............]  Step: 66ms | Tot: 11s466ms | Loss: 12.247 | Acc: 62.479% (14875/2380 186/39 187/391 191/391 208/39 219/39 267/391 ==========>.......]  Step: 69ms | Tot: 17s119ms | Loss: 12.238 | Acc: 62.785% (22261/3545 277/391 278/391 ============>.......]  Step: 65ms | Tot: 17s249ms | Loss: 12.237 | Acc: 62.808% (22430/3571 279/391 ...]  Step: 65ms | Tot: 17s384ms | Loss: 12.238 | Acc: 62.781% (22581/3596 281/391 =======>......]  Step: 66ms | Tot: 17s845ms | Loss: 12.236 | Acc: 62.828% (23161/3686 288/391 289/391 290/391 291/391 293/391 ]  Step: 68ms | Tot: 18s874ms | Loss: 12.236 | Acc: 62.747% (24416/3891 304/391   Step: 61ms | Tot:

 [========================>]  Step: 58ms | Tot: 22s640ms | Loss: 11.935 | Acc: 67.326% (33663/5000 391/391 1 ........]  Step: 63ms | Tot: 1s968ms | Loss: 11.874 | Acc: 68.549% (3071/448 35/391 .........]  Step: 70ms | Tot: 7s94ms | Loss: 11.943 | Acc: 66.927% (10280/1536 120/391 >.................]  Step: 62ms | Tot: 7s157ms | Loss: 11.944 | Acc: 66.923% (10365/1548 121/391 ..........]  Step: 48ms | Tot: 8s473ms | Loss: 11.956 | Acc: 66.643% (12369/1856 145/39 173/391 179/39 284/39 308/391 309/39 376/391 380/391 
 [========================>]  Step: 44ms | Tot: 3s601ms | Loss: 14.114 | Acc: 1.000% (100/1000 79/79 
Saving..
Sparsity:  [1.0, 1.0, 1.0, 1.0, 1.0]
SNRS:  [tensor(0.8119, device='cuda:0'), tensor(0.7566, device='cuda:0'), tensor(0.7905, device='cuda:0'), tensor(0.8021, device='cuda:0'), tensor(0.7513, device='cuda:0'), tensor(1.6557, device='cuda:0'), tensor(1.3512, device='cuda:0')]

Epoch: 8
 [========================>]  Step: 66ms | Tot: 20s565ms | Loss: 11.915 | Acc: 67.26

 [========================>]  Step: 65ms | Tot: 24s855ms | Loss: 11.861 | Acc: 67.782% (33891/5000 391/391 ...........]  Step: 69ms | Tot: 770ms | Loss: 11.702 | Acc: 71.635% (1192/166 13/39 20/391 .......]  Step: 67ms | Tot: 1s353ms | Loss: 11.722 | Acc: 71.165% (2004/281 22/3 23/391 ............]  Step: 66ms | Tot: 1s991ms | Loss: 11.760 | Acc: 70.508% (2888/409 32/391 .................]  Step: 68ms | Tot: 2s446ms | Loss: 11.801 | Acc: 69.752% (3482/499 39/39 53/391 ...................]  Step: 66ms | Tot: 3s556ms | Loss: 11.828 | Acc: 68.862% (4936/7 56/391 ....................]  Step: 62ms | Tot: 4s445ms | Loss: 11.841 | Acc: 68.661% (6152/896 70/39 73/39 101/391 .................]  Step: 67ms | Tot: 6s543ms | Loss: 11.850 | Acc: 68.272% (9001/1318 103/391 ............]  Step: 65ms | Tot: 6s674ms | Loss: 11.850 | Acc: 68.222% (9169/1344 105/391 106/39 112/391 113/391 114/39 131/391  143/391 ===>...............]  Step: 68ms | Tot: 9s629ms | Loss: 11.872 | Acc: 67.679% (13081/1932 151

 [========================>]  Step: 65ms | Tot: 21s58ms | Loss: 11.847 | Acc: 67.864% (33932/5000 391/391  ...]  Step: 68ms | Tot: 1s477ms | Loss: 11.687 | Acc: 72.062% (2306/320 25/391 =>.......................]  Step: 67ms | Tot: 1s544ms | Loss: 11.693 | Acc: 71.785% (2389/332 26/391 41/391 ...............]  Step: 66ms | Tot: 3s980ms | Loss: 11.823 | Acc: 68.798% (5724/832 65/39 67/391 ..................]  Step: 52ms | Tot: 4s423ms | Loss: 11.824 | Acc: 68.739% (6423/934 73/391 ....................]  Step: 51ms | Tot: 4s475ms | Loss: 11.825 | Acc: 68.761% (6513/947 74/39 101/391 ..........]  Step: 54ms | Tot: 6s637ms | Loss: 11.840 | Acc: 68.281% (10051/1472 115/391 .............]  Step: 54ms | Tot: 7s49ms | Loss: 11.841 | Acc: 68.166% (10732/1574 123/391 ========>................]  Step: 48ms | Tot: 7s302ms | Loss: 11.845 | Acc: 67.963% (11135/1638 128/391 .........]  Step: 46ms | Tot: 7s504ms | Loss: 11.846 | Acc: 67.951% (11481/1689 132/39 133/39 134/39 135/391 .]  Step: 44ms | To

 [========================>]  Step: 64ms | Tot: 25s395ms | Loss: 11.844 | Acc: 67.918% (33959/5000 391/391 91 61/391 62/39 85/39 92/391 ................]  Step: 62ms | Tot: 7s11ms | Loss: 11.844 | Acc: 67.865% (9816/1446 113/39 121/39 134/391 =>................]  Step: 65ms | Tot: 8s405ms | Loss: 11.844 | Acc: 67.940% (11740/1728 135/391 137/391 ====>................]  Step: 69ms | Tot: 8s604ms | Loss: 11.845 | Acc: 67.969% (12006/1766 138/391 142/391 =>...............]  Step: 68ms | Tot: 9s264ms | Loss: 11.846 | Acc: 67.890% (12861/1894 148/391 ...]  Step: 60ms | Tot: 9s395ms | Loss: 11.847 | Acc: 67.849% (13027/1920 150/39 151/391 .......]  Step: 63ms | Tot: 9s875ms | Loss: 11.847 | Acc: 67.829% (13631/2009 157/39 165/391 ==>..............]  Step: 70ms | Tot: 10s612ms | Loss: 11.846 | Acc: 67.825% (14585/2150 168/39 179/391 181/391 187/391 .......]  Step: 62ms | Tot: 12s145ms | Loss: 11.851 | Acc: 67.736% (16560/2444 191/391 201/391 217/39 221/391 .......]  Step: 61ms | Tot: 14s696ms

 [========================>]  Step: 60ms | Tot: 23s925ms | Loss: 11.847 | Acc: 67.768% (33884/5000 391/391 ..........]  Step: 68ms | Tot: 1s705ms | Loss: 11.741 | Acc: 70.508% (2527/358 28/39 36/391 ................]  Step: 64ms | Tot: 5s156ms | Loss: 11.834 | Acc: 68.301% (7606/1113 87/391 88/391 .................]  Step: 66ms | Tot: 5s550ms | Loss: 11.838 | Acc: 68.246% (8124/1190 93/391 ............]  Step: 70ms | Tot: 6s579ms | Loss: 11.845 | Acc: 67.926% (9564/1408 110/39 127/391 ===>................]  Step: 69ms | Tot: 7s728ms | Loss: 11.848 | Acc: 67.731% (11097/1638 128/39 130/391 ==>................]  Step: 68ms | Tot: 8s183ms | Loss: 11.848 | Acc: 67.714% (11701/1728 135/391 ............]  Step: 63ms | Tot: 8s315ms | Loss: 11.848 | Acc: 67.741% (11879/1753 137/39 154/391 156/39 179/39 182/391 ====>.............]  Step: 69ms | Tot: 11s450ms | Loss: 11.860 | Acc: 67.440% (16056/2380 186/39 187/391 ........]  Step: 63ms | Tot: 13s531ms | Loss: 11.862 | Acc: 67.272% (19030/2828 2

 [========================>]  Step: 64ms | Tot: 21s35ms | Loss: 11.852 | Acc: 67.640% (33820/5000 391/391  41/391 171/39 367/39 368/391 ==========>.]  Step: 53ms | Tot: 19s673ms | Loss: 11.856 | Acc: 67.554% (31907/4723 369/391 389/391 


KeyboardInterrupt: 

In [ ]:

for epoch in range(300,400):
    SBP_net_train(epoch,sbp_equal_net,optimizer=equal_optimizer,criterion=nn.CrossEntropyLoss(),lr_adjust=learning_rate_calc)
    SBP_net_test(epoch,sbp_equal_net)

## Restore and Train

In [18]:
best_sbp_acc = best_base_acc = 0 #reset best accuracy to save after running SBP
equal_weights = [1,1,1,1,1,1,1]
sbp_equal_net = SBPConv_AlexNet(cfg,kl_weights=equal_weights).to(device)

In [19]:
net_dict = torch.load('./checkpoint/sbp_ckpt.pth')
sbp_equal_net.load_state_dict(net_dict['net'])
best_acc = net_dict['best_acc']

RuntimeError: Error(s) in loading state_dict for SBPConv_AlexNet:
	Missing key(s) in state_dict: "block1.conv1.log_sigma", "block1.conv1.mu", "block1.conv1.weight", "block1.bn1.weight", "block1.bn1.bias", "block1.bn1.running_mean", "block1.bn1.running_var", "block2.conv1.log_sigma", "block2.conv1.mu", "block2.conv1.weight", "block2.bn1.weight", "block2.bn1.bias", "block2.bn1.running_mean", "block2.bn1.running_var", "block3.conv1.log_sigma", "block3.conv1.mu", "block3.conv1.weight", "block3.bn1.weight", "block3.bn1.bias", "block3.bn1.running_mean", "block3.bn1.running_var", "block4.conv1.log_sigma", "block4.conv1.mu", "block4.conv1.weight", "block4.bn1.weight", "block4.bn1.bias", "block4.bn1.running_mean", "block4.bn1.running_var", "block5.conv1.log_sigma", "block5.conv1.mu", "block5.conv1.weight", "block5.bn1.weight", "block5.bn1.bias", "block5.bn1.running_mean", "block5.bn1.running_var", "lsbp1.weight", "lsbp1.bias", "lsbp1.log_sigma", "lsbp1.mu", "lsbp2.weight", "lsbp2.bias", "lsbp2.

In [31]:
net_dict['net'].keys()

odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.2.running_mean', 'features.2.running_var', 'features.2.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.6.weight', 'features.6.bias', 'features.6.running_mean', 'features.6.running_var', 'features.6.num_batches_tracked', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'features.10.running_mean', 'features.10.running_var', 'features.10.num_batches_tracked', 'features.11.weight', 'features.11.bias', 'features.13.weight', 'features.13.bias', 'features.13.running_mean', 'features.13.running_var', 'features.13.num_batches_tracked', 'features.14.weight', 'features.14.bias', 'features.16.weight', 'features.16.bias', 'features.16.running_mean', 'features.16.running_var', 'features.16.num_batches_tracked', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])

In [27]:
list(sbp_equal_net.parameters())

[Parameter containing:
 tensor([-5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5.,
         -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5.,
         -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5.,
         -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5.,
         -5., -5., -5., -5., -5., -5., -5., -5.], device='cuda:0',
        requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[[[-0.1025, -0.4885,  0.4237],
           [-0.0737,  0.5720,  0.1400],
           [ 0.4433,  0.4696,  0.2519]],
 
          [[ 0.5657, -0.2519,  0.2215],
           [-0.1572,  0.09

In [32]:
for epoch in range(0,300):
    SBP_net_train(epoch,sbp_equal_net,optimizer=equal_optimizer,criterion=nn.CrossEntropyLoss())
    SBP_net_test(epoch,sbp_equal_net)

In [33]:
lr_x

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [40]:
for epoch in range(0,300):
    SBP_net_train(epoch,sbp_net,optimizer=optimizer,criterion=nn.CrossEntropyLoss())
    SBP_net_test(epoch,sbp_net)


Epoch: 30
 [========================>]  Step: 64ms | Tot: 21s394ms | Loss: 6190.482 | Acc: 41.866% (20933/5000 391/391 ................]  Step: 49ms | Tot: 631ms | Loss: 6212.230 | Acc: 42.132% (755/179 14/391 =>.......................]  Step: 50ms | Tot: 1s244ms | Loss: 6211.542 | Acc: 41.046% (1366/332 26/39 28/39 31/391 34/39 42/391 ..............]  Step: 51ms | Tot: 2s580ms | Loss: 6210.072 | Acc: 40.850% (2719/665 52/391 ................]  Step: 50ms | Tot: 2s630ms | Loss: 6210.016 | Acc: 40.846% (2771/678 53/391 ....................]  Step: 57ms | Tot: 2s831ms | Loss: 6209.781 | Acc: 40.954% (2988/729 57/391 104/391 ======>.............]  Step: 50ms | Tot: 9s7ms | Loss: 6203.003 | Acc: 41.680% (9283/2227 174/39 224/391 ==============>..........]  Step: 50ms | Tot: 11s632ms | Loss: 6200.061 | Acc: 41.736% (12020/2880 225/391 ........]  Step: 50ms | Tot: 11s683ms | Loss: 6200.003 | Acc: 41.738% (12074/2892 226/391 =====>...]  Step: 48ms | Tot: 18s549ms | Loss: 6193.312 | Acc: 41.7

### Resume Training

In [23]:
print('==> Resuming from checkpoint..')
assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('./checkpoint/ckpt.pth')
sbp_net.load_state_dict(checkpoint['net'])
best_acc = checkpoint['best_acc']
start_epoch = 20 # set manually from previous training. 

==> Resuming from checkpoint..


In [29]:
x = torch.randn(2,3,32,32)
sbp_net.eval()
sbp_net(x)
print(sbp_net.features[0].sbp.mask)
print(sbp_net.features[0].sbp.multiplicator)

SNR:  tensor(7289.9297)
MASK:  tensor(64.)
SNR:  tensor(21809.4375)
MASK:  tensor(192.)
SNR:  tensor(43668.1445)
MASK:  tensor(384.)
SNR:  tensor(29090.8965)
MASK:  tensor(256.)
SNR:  tensor(29085.9785)
MASK:  tensor(256.)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([[[0.9588, 0.9588, 0.9588,  ..., 0.9588, 0.9588, 0.9588],
         [0.9588, 0.9588, 0.9588,  ..., 0.9588, 0.9588, 0.9588],
         [0.9588, 0.9588, 0.9588,  ..., 0.9588, 0.9588, 0.9588],
         ...,
         [0.9588, 0.9588, 0.9588,  ..., 0.9588, 0.9588, 0.9588],
         [0.9588, 0.9588, 0.9588,  ..., 0.9588, 0.9588, 0.9588],
         [0.9588, 0.9588, 0.9588,  ..., 0.9588, 0.9588, 0.9588]],

        [[0.9588, 0.9588, 0.9588,  ..., 0.9588, 0.9588, 0.9588],
         [0.9588,

In [45]:
print(best_acc)

50.39


In [ ]:
for epoch in range(20,45):
    SBP_net_train(epoch,sbp_net)
    SBP_net_test(epoch,sbp_net)


Epoch: 20
 [========================>]  Step: 293ms | Tot: 1m36s | Loss: 7.817 | Acc: 60.584% (30292/5000 391/391 1  
 [========================>]  Step: 93ms | Tot: 7s527ms | Loss: 2.006 | Acc: 50.180% (5018/1000 79/79 1/79 

Epoch: 21
 [========================>]  Step: 274ms | Tot: 1m37s | Loss: 7.805 | Acc: 60.658% (30329/5000 391/391 1  
 [========================>]  Step: 104ms | Tot: 7s87ms | Loss: 2.002 | Acc: 50.010% (5001/1000 79/79 .....................]  Step: 97ms | Tot: 864ms | Loss: 2.126 | Acc: 48.698% (561/115 9/79 

Epoch: 22
 [========================>]  Step: 250ms | Tot: 1m37s | Loss: 7.783 | Acc: 61.118% (30559/5000 391/391 1  
 [========================>]  Step: 104ms | Tot: 7s258ms | Loss: 1.998 | Acc: 50.380% (5038/1000 79/79 

Epoch: 23
 [========================>]  Step: 257ms | Tot: 1m37s | Loss: 7.768 | Acc: 61.090% (30545/5000 391/391 1  
 [========================>]  Step: 106ms | Tot: 6s860ms | Loss: 2.003 | Acc: 50.380% (5038/1000 79/79 

Epoch: 24
 [=

### Get Mask & Prune Network

[SBP_Block(
   (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (sbp): SBP_layer()
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (rel): ReLU(inplace=True)
 ),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 SBP_Block(
   (conv1): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (sbp): SBP_layer()
   (bn1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (rel): ReLU(inplace=True)
 ),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 SBP_Block(
   (conv1): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (sbp): SBP_layer()
   (bn1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (rel): ReLU(inplace=True)
 ),
 SBP_Block(
   (conv1): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (sbp): SBP_layer()
   (bn1): BatchNorm2d(2

In [70]:
def cal_importance_0(net, l_id, num_stop=100):
    layer_weights = net.features[0].sbp.weight.data
    imp_corr_bn = layer_weights.abs().sum(dim=(1,2,3))        
    neuron_order = [np.linspace(0, imp_corr_bn.shape[0]-1, imp_corr_bn.shape[0]), imp_corr_bn]
    return neuron_order

In [1]:
def snr_truncated_log_normal(mu, sigma, a, b):
    alpha = (a - mu)/sigma
    beta = (b - mu)/sigma
    z = phi(beta) - phi(alpha)
    ratio = erfcx((sigma-beta)/(2 ** 0.5))*torch.exp((b-mu)-beta**2/2.0)
    ratio = ratio - erfcx((sigma-alpha)/2 ** 0.5)*torch.exp((a-mu)-alpha**2/2.0)
    denominator = 2*z*erfcx((2.0*sigma-beta)/2 ** 0.5)*torch.exp(2.0*(b-mu)-beta**2/2.0)
    denominator = denominator - 2*z*erfcx((2.0*sigma-alpha)/(2 ** 0.5))*torch.exp(2.0*(a-mu)-alpha**2/2.0)
    denominator = denominator - ratio**2
    ratio = ratio/torch.sqrt(1e-8 + denominator)
    print(denominator)
    return ratio

In [2]:
## get the sbp layer mask
def get_mask(mu,sigma,min_log=-20, max_log=0):
    multiplicator = SBP_utils.mean_truncated_log_normal_reduced(mu, sigma, min_log, max_log)
    print("multiplicator: ",multiplicator)
    snr = snr_truncated_log_normal(mu, sigma, min_log, max_log)
    print("snr: ", snr)
    mask = snr
    
    mask[snr <= 1.0] = 0.0
    mask[snr > 1.0] = 1.
    
    print("mask: ",mask)
    multiplicator = multiplicator * mask
    
    return mask, multiplicator

In [107]:
sbp_net.features[0].sbp.log_sigma.detach()

tensor([-4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441])

In [3]:
mask_list = []
multiplicator_list = []
for i in sbp_net.features:
    if not (isinstance(i,nn.MaxPool2d)):
        #print(i)
        mu = i.sbp.mu.detach()
        sigma = i.sbp.log_sigma.detach()
        mask,multiplicator = get_mask(mu,sigma)
        mask_list.append(mask)
        multiplicator_list.append(multiplicator)

NameError: name 'sbp_net' is not defined

In [118]:
x = torch.randn(2,3,32,32)
sbp_net.eval()
y = sbp_net.forward(x)
print(y,y.size())


tensor([[ -6.5035,  -8.3538,  -6.6634, -10.2934, -17.9784,  -5.0302, -11.5280,
         -10.6385,  -1.8071,  -4.3939,   4.3780, -12.2435, -13.1021, -10.5908,
          -7.5355, -12.7850,   2.3540, -12.5417,  -9.5258, -16.9257,  -8.0328,
         -16.6011,   4.9747,  -3.3861, -12.9907,  -4.4500,  -5.9589, -14.0933,
          -4.2610, -14.5732, -16.0110, -13.5321,  -3.9859,  -2.0124, -13.0010,
          -9.6966,  -9.1011,  -3.9229, -16.7927,   8.4844,  -8.6888, -13.6158,
          -8.8288,  -9.8294, -15.8931,  -3.8235, -12.4232,  -6.1532, -10.6135,
         -11.1450,  -8.6385,  -7.6694, -15.9593,  -3.3235,  -4.4470, -18.4937,
         -20.6754,  -0.6448, -18.1565,  -9.3899, -13.1982,   7.3392,  -3.7231,
          -9.8469, -13.8983,  -7.2742, -11.2102, -11.6250, -19.0181, -17.3108,
           3.8214, -17.8009, -10.2750, -17.4043, -15.8597, -20.3146, -11.4058,
         -14.2033,   1.2470,  -9.8833, -10.9950,  -7.2665,  -4.5578,  -1.9062,
          -7.2311, -17.2258,  -2.7540,  -8.5210,  -3

In [137]:
sbp_net.eval()
out_1 = sbp_net.features[0].conv1(x)
print(out.shape)
out_2 = sbp_net.features[0].sbp(out_1)
mu = sbp_net.features[0].sbp.mu.detach()
sigma = sbp_net.features[0].sbp.log_sigma.detach()
print(sbp_net.features[0].sbp.mu)
print(sbp_net.features[0].sbp.log_sigma)
print(get_mask(mu,sigma))
#print(out_1)

torch.Size([2, 64, 16, 16])
Parameter containing:
tensor([-0.0501, -0.0499, -0.0500, -0.0501, -0.0501, -0.0500, -0.0497, -0.0501,
        -0.0500, -0.0500, -0.0502, -0.0500, -0.0501, -0.0500, -0.0498, -0.0500,
        -0.0501, -0.0509, -0.0499, -0.0500, -0.0500, -0.0500, -0.0501, -0.0500,
        -0.0501, -0.0500, -0.0498, -0.0500, -0.0500, -0.0500, -0.0500, -0.0500,
        -0.0500, -0.0499, -0.0501, -0.0500, -0.0501, -0.0500, -0.0501, -0.0499,
        -0.0500, -0.0500, -0.0497, -0.0501, -0.0500, -0.0501, -0.0495, -0.0500,
        -0.0501, -0.0499, -0.0500, -0.0500, -0.0500, -0.0501, -0.0500, -0.0505,
        -0.0500, -0.0500, -0.0500, -0.0498, -0.0502, -0.0500, -0.0499, -0.0513],
       requires_grad=True)
Parameter containing:
tensor([-4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441, -4.6441,
        -4.6441, -4.

tensor([[[[-4.3465e-01, -4.3465e-01, -1.7600e-01,  ..., -4.3465e-01,
            1.7735e+00, -4.3465e-01],
          [ 1.3551e+00,  5.0437e-01, -4.3465e-01,  ..., -4.3465e-01,
           -3.1233e-01,  1.1398e+00],
          [-4.3465e-01, -4.3465e-01, -4.3465e-01,  ...,  5.5210e-01,
           -4.3465e-01, -4.3465e-01],
          ...,
          [ 2.9487e+00, -4.3465e-01, -4.3465e-01,  ..., -4.3465e-01,
            5.1174e-01, -4.3465e-01],
          [-4.3465e-01, -3.3155e-01, -4.3465e-01,  ..., -4.3465e-01,
           -4.3465e-01, -4.3465e-01],
          [-4.3465e-01,  1.4996e+00,  3.1659e+00,  ..., -4.3465e-01,
            9.4603e-01,  3.1426e+00]],

         [[-5.0862e-01,  1.2093e+00, -5.0862e-01,  ...,  9.4067e-02,
           -5.0862e-01, -5.0862e-01],
          [-5.0862e-01,  3.7695e-01,  4.3916e-01,  ..., -5.0862e-01,
            3.3940e+00,  4.3031e-01],
          [ 2.3318e+00,  1.4081e+00,  4.7294e+00,  ..., -5.0862e-01,
           -5.0862e-01, -5.0862e-01],
          ...,
     

In [71]:
cal_importance_0(sbp_net,l_id=1,num_stop=100)

AttributeError: 'SBP_layer' object has no attribute 'weight'

### PTFLOPS

In [ ]:
pt

In [29]:
from ptflops import get_model_complexity_info

with torch.cuda.device(-1):
    flops, params = get_model_complexity_info(sbp_net, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
    print('{:<30}  {:<8}'.format('Computational complexity: ', flops))

SBPConv_AlexNet(
  20.516 M, 100.000% Params, 0.001 GMac, 100.000% MACs, 
  (block1): SBP_ConvBlock(
    0.002 M, 0.010% Params, 0.0 GMac, 8.978% MACs, 
    (conv1): Conv2d_SBP(0.002 M, 0.009% Params, 0.0 GMac, 0.000% MACs, )
    (bn1): BatchNorm2d(0.0 M, 0.001% Params, 0.0 GMac, 5.986% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 2.993% MACs, inplace=True)
  )
  (mp1): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 2.993% MACs, kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block2): SBP_ConvBlock(
    0.112 M, 0.544% Params, 0.0 GMac, 5.155% MACs, 
    (conv1): Conv2d_SBP(0.111 M, 0.542% Params, 0.0 GMac, 0.000% MACs, )
    (bn1): BatchNorm2d(0.0 M, 0.002% Params, 0.0 GMac, 3.437% MACs, 192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 1.718% MACs, inplace=True)
  )
  (mp2): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 1.718% MACs

In [39]:
with torch.cuda.device(-1):
    #flops, params = get_model_complexity_info(sbp_net, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
    flops, params = get_model_complexity_info(best_net, (3, 32, 32), as_strings=True, print_per_layer_stat=True)

AlexNet(
  20.498 M, 100.000% Params, 0.044 GMac, 100.000% MACs, 
  (features): Sequential(
    2.254 M, 10.996% Params, 0.025 GMac, 58.072% MACs, 
    (0): Conv2d(0.002 M, 0.009% Params, 0.0 GMac, 1.054% MACs, 3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.038% MACs, inplace=True)
    (2): BatchNorm2d(0.0 M, 0.001% Params, 0.0 GMac, 0.075% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 0.038% MACs, kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(0.111 M, 0.540% Params, 0.005 GMac, 12.476% MACs, 64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.022% MACs, inplace=True)
    (6): BatchNorm2d(0.0 M, 0.002% Params, 0.0 GMac, 0.043% MACs, 192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 0.022% MAC

In [ ]:
sbp_net.

### Flop weighted importance

In [28]:
### importance for SBP Net

l_imp = {}

for conv_ind in [0, 2, 4, 5, 6]:
    l_imp.update({conv_ind: net.features[conv_ind].bn1.bias.shape[0]})
## do not need to update the classifer indices b/c no blocks 
for lin_ind in [1, 4]:
    l_imp.update({lin_ind: net.classifier[lin_ind].bias.shape[0]})
    
normalizer = 0
for key, val in l_imp.items():
    normalizer += val
for key, val in l_imp.items():
    l_imp[key] = val / normalizer

In [29]:
optimizer = optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-08, weight_decay=5e-4, amsgrad=False)

In [30]:
net = SBP_AlexNet(cfg)
net_ortho = SBP_AlexNet(cfg)

In [31]:
for epoch in range(30):
    SBP_net_train(epoch,net)
    SBP_net_test(epoch,net)
    


Epoch: 0
 [========================>]  Step: 261ms | Tot: 1m35s | Loss: 11.952 | Acc: 1.048% (524/5000 391/391 1  
 [========================>]  Step: 89ms | Tot: 6s501ms | Loss: 4.621 | Acc: 1.310% (131/1000 79/79 
Saving..

Epoch: 1
 [========================>]  Step: 266ms | Tot: 1m36s | Loss: 11.952 | Acc: 0.992% (496/5000 391/391 1  
 [========================>]  Step: 97ms | Tot: 6s579ms | Loss: 4.621 | Acc: 1.310% (131/1000 79/79 

Epoch: 2
 [========================>]  Step: 262ms | Tot: 1m37s | Loss: 11.952 | Acc: 1.076% (538/5000 391/391 1  
 [========================>]  Step: 101ms | Tot: 6s502ms | Loss: 4.621 | Acc: 1.370% (137/1000 79/79 
Saving..

Epoch: 3
 [========================>]  Step: 325ms | Tot: 1m47s | Loss: 11.952 | Acc: 1.102% (551/5000 391/391 1  
 [========================>]  Step: 88ms | Tot: 6s200ms | Loss: 4.621 | Acc: 1.320% (132/1000 79/79 

Epoch: 4


KeyboardInterrupt: 

In [24]:
for epoch in range(30):
    SBP_net_train_ortho(epoch,net_ortho)
    w_diag(net_ortho)
    SBP_net_test_ortho(epoch,net_ortho)


Epoch: 0
 [========================>]  Step: 154ms | Tot: 1m4s | Loss: 11.956 | Acc: 1.018% (509/5000 391/391 91  

Epoch: 0
 [========================>]  Step: 714ms | Tot: 4m20s | Loss: 3634.107 | Acc: 0.970% (485/5000 391/391 1 
angle_cost:  283.252374375


TypeError: net_test() takes 1 positional argument but 2 were given

# Inner product training

In [16]:
alex_net = AlexNet(cfg)
print(alex_net)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Conv2d(2

In [18]:
alex_net.features[2].bias

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)

In [ ]:
net_ortho = AlexNet(cfg).to(device)
net_dict = torch.load('./ortho_checkpoint/ckpt.pth')
net_ortho.load_state_dict(net_dict['net'])
best_acc_ortho = net_dict['best_acc']

In [25]:
### importance for SBP Net

l_imp = {}

for conv_ind in [0, 2, 4, 5, 6]:
    l_imp.update({conv_ind: net.features[conv_ind].bn1.bias.shape[0]})
## do not need to update the classifer indices b/c no blocks 
for lin_ind in [1, 4]:
    l_imp.update({lin_ind: net.classifier[lin_ind].bias.shape[0]})
    
normalizer = 0
for key, val in l_imp.items():
    normalizer += val
for key, val in l_imp.items():
    l_imp[key] = val / normalizer

In [ ]:
### normal AlexNet
l_imp = {}

for conv_ind in [2, 6, 10, 13, 16]:
    l_imp.update({conv_ind: net.features[conv_ind].bias.shape[0]})
    
for lin_ind in [1, 4]:
    l_imp.update({lin_ind: net.classifier[lin_ind].bias.shape[0]})
    
normalizer = 0
for key, val in l_imp.items():
    normalizer += val
for key, val in l_imp.items():
    l_imp[key] = val / normalizer

In [18]:
def net_test_ortho(epoch):
    global best_acc_ortho
    net_ortho.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net_ortho(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    print(acc)
    if acc > best_acc_ortho:
        print('Saving..')
        state = {
            'net': net_ortho.state_dict(),
            'best_acc': acc
        }
        if not os.path.isdir('ortho_checkpoint'):
            os.mkdir('ortho_checkpoint')
        torch.save(state, './ortho_checkpoint/ckpt.pth')
        best_acc_ortho = acc

In [20]:

for conv_ind in [6, 10, 13, 16]:
    print(alex_net.features[conv_ind-2])

Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [19]:
def SBP_net_train_ortho(epoch,net_ortho):
    print('\nEpoch: %d' % epoch)
    net_ortho.train()
    correct = 0
    total = 0
    running_loss = 0.0
    angle_cost = 0.0
            
    for batch_idx, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs, kl = net_ortho(inputs)
        L_angle = 0
        
        ### Conv_ind == 0 ###
        w_mat = net_ortho.features[0].conv1.weight
        w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
        b_mat = net_ortho.features[0].conv1.bias
        b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))
        params = torch.cat((w_mat1, b_mat1), dim=1)
        angle_mat = torch.matmul(torch.t(params), params) - torch.eye(params.shape[1]).to(device)
        L_angle += (l_imp[2])*(angle_mat).norm(1) #.norm().pow(2))

        ### Conv_ind != 0 ###
        for conv_ind in [2, 4, 5, 6]:
            w_mat = net_ortho.features[conv_ind].conv1.weight
            w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
            b_mat = net_ortho.features[conv_ind].conv1.bias
            b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))
            params = torch.cat((w_mat1, b_mat1), dim=1)
            angle_mat = torch.matmul(params, torch.t(params)) - torch.eye(w_mat.shape[0]).to(device)            
            L_angle += (l_imp[conv_ind])*(angle_mat).norm(1) #.norm().pow(2))
    
        ### lin_ind = 1 ###        
        w_mat = net_ortho.classifier[1].weight
        w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
        b_mat = net_ortho.classifier[1].bias
        b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))            
        params = torch.cat((w_mat1, b_mat1), dim=1)
        angle_mat = torch.matmul(torch.t(params), params) - torch.eye(params.shape[1]).to(device)
        L_angle += (l_imp[1])*(angle_mat).norm(1) #.norm().pow(2))
        
        ### lin_ind = 4 ###        
        w_mat = net_ortho.classifier[4].weight
        w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
        b_mat = net_ortho.classifier[4].bias
        b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))            
        params = torch.cat((w_mat1, b_mat1), dim=1)
        angle_mat = torch.matmul(params, torch.t(params)) - torch.eye(params.shape[0]).to(device)
        L_angle += (l_imp[4])*(angle_mat).norm(1) #.norm().pow(2))        
        
        Lc = criterion(outputs, labels)
        loss = (1e-1)*(L_angle) + Lc + kl #from the sparsity inducer
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        angle_cost += (L_angle).item()
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (running_loss/(batch_idx+1), 100.*correct/total, correct, total))
    
    print("angle_cost: ", angle_cost/total)

In [ ]:
def net_train_ortho(epoch):
    print('\nEpoch: %d' % epoch)
    net_ortho.train()
    correct = 0
    total = 0
    running_loss = 0.0
    angle_cost = 0.0
            
    for batch_idx, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net_ortho(inputs)
        L_angle = 0
        
        ### Conv_ind == 0 ###
        w_mat = net_ortho.features[0].weight
        w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
        b_mat = net_ortho.features[0].bias
        b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))
        params = torch.cat((w_mat1, b_mat1), dim=1)
        angle_mat = torch.matmul(torch.t(params), params) - torch.eye(params.shape[1]).to(device)
        L_angle += (l_imp[2])*(angle_mat).norm(1) #.norm().pow(2))

        ### Conv_ind != 0 ###
        for conv_ind in [6, 10, 13, 16]:
            w_mat = net_ortho.features[conv_ind-2].weight
            w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
            b_mat = net_ortho.features[conv_ind-2].bias
            b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))
            params = torch.cat((w_mat1, b_mat1), dim=1)
            angle_mat = torch.matmul(params, torch.t(params)) - torch.eye(w_mat.shape[0]).to(device)            
            L_angle += (l_imp[conv_ind])*(angle_mat).norm(1) #.norm().pow(2))
    
        ### lin_ind = 1 ###        
        w_mat = net_ortho.classifier[1].weight
        w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
        b_mat = net_ortho.classifier[1].bias
        b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))            
        params = torch.cat((w_mat1, b_mat1), dim=1)
        angle_mat = torch.matmul(torch.t(params), params) - torch.eye(params.shape[1]).to(device)
        L_angle += (l_imp[1])*(angle_mat).norm(1) #.norm().pow(2))
        
        ### lin_ind = 4 ###        
        w_mat = net_ortho.classifier[4].weight
        w_mat1 = (w_mat.reshape(w_mat.shape[0],-1))
        b_mat = net_ortho.classifier[4].bias
        b_mat1 = (b_mat.reshape(b_mat.shape[0],-1))            
        params = torch.cat((w_mat1, b_mat1), dim=1)
        angle_mat = torch.matmul(params, torch.t(params)) - torch.eye(params.shape[0]).to(device)
        L_angle += (l_imp[4])*(angle_mat).norm(1) #.norm().pow(2))        
        
        Lc = criterion(outputs, labels)
        loss = (1e-1)*(L_angle) + Lc
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        angle_cost += (L_angle).item()
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (running_loss/(batch_idx+1), 100.*correct/total, correct, total))
    
    print("angle_cost: ", angle_cost/total)

In [22]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()

In [15]:
SBP_net_ortho = SBP_AlexNet(cfg).to(device)

In [23]:
optimizer = optim.Adam(SBP_net_ortho.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [17]:
SBP_net_ortho

SBP_AlexNet(
  (block1): SBP_Block(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (sbp): SBP_layer()
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(inplace=True)
  )
  (mp1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block2): SBP_Block(
    (conv1): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (sbp): SBP_layer()
    (bn1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(inplace=True)
  )
  (mp2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block3): SBP_Block(
    (conv1): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (sbp): SBP_layer()
    (bn1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(inplace=True)
  )
  (block4): SBP_Block(
    (conv1): Conv2d(384, 256, kernel_size=

In [25]:
for epoch in range(2):
    #SBP_net_train(epoch,SBP_net_ortho)
    SBP_net_train_ortho(epoch,SBP_net_ortho)
    #net_test_ortho(epoch)
    #w_diag(net_ortho)

NameError: name 'SBP_net_ortho' is not defined

In [ ]:
# PATH = './w_decorr/base_params/wnet_base_2.pth'
# torch.save(net.state_dict(), PATH)

# Importance analysis

### Layer index

In [35]:
alex_net

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Conv2d(2

In [37]:
SBP_net_ortho

SBP_AlexNet(
  (block1): SBP_Block(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (sbp): SBP_layer()
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(inplace=True)
  )
  (mp1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block2): SBP_Block(
    (conv1): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (sbp): SBP_layer()
    (bn1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(inplace=True)
  )
  (mp2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block3): SBP_Block(
    (conv1): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (sbp): SBP_layer()
    (bn1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rel): ReLU(inplace=True)
  )
  (block4): SBP_Block(
    (conv1): Conv2d(384, 256, kernel_size=

In [39]:
l_index = 4
layer_id = 'bn'

### Correlated Net

In [ ]:
net = AlexNet(cfg).to(device)
net_dict = torch.load('./checkpoint/ckpt.pth')
net.load_state_dict(net_dict['net'])
net = net.eval()

In [40]:
#changed for SBP_Aexnet
weight_base = net.features[l_index].bn1.weight.data.clone().detach()
bias_base = net.features[l_index].bn1.bias.data.clone().detach()

In [41]:
loss_base_corr = 0
num_stop = 0
for epoch in range(1):
#     for i, data in enumerate(testloader, 0):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs, kl = net(inputs)
        loss = criterion(outputs, labels) + kl
        loss_base_corr += loss.item()
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
loss_base_corr = loss_base_corr**2

In [46]:
## changed for SBP_Alexnet

loss_mat_corr = torch.zeros(weight_base.shape[0])

for n_index in tqdm(range(weight_base.shape[0])): 
    num_stop = 0
    print(n_index)
    running_loss = 0.0

    net.features[l_index].bn1.weight.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
    net.features[l_index].bn1.bias.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
    
#     for i, data in enumerate(testloader, 0):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        outputs, kl = net(inputs)

        loss = (criterion(outputs, labels)) + kl

        running_loss += loss.item()
        
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
            
    loss_mat_corr[n_index] = running_loss**2
    
    net.features[l_index].bn1.weight.data = weight_base.clone().detach()
    net.features[l_index].bn1.bias.data = bias_base.clone().detach()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [44]:
#normal alex net w/ no KL loss
loss_base_corr = 0
num_stop = 0
for epoch in range(1):
#     for i, data in enumerate(testloader, 0):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss_base_corr += loss.item()
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
loss_base_corr = loss_base_corr**2

In [ ]:
### SBP_Alexnet
### Change bn1 to access other layers in the block.
loss_mat_corr = torch.zeros(weight_base.shape[0])

for n_index in tqdm(range(weight_base.shape[0])): 
    num_stop = 0
    print(n_index)
    running_loss = 0.0

    net.features[l_index].bn1.weight.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
    net.features[l_index].bn1.bias.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
    
#     for i, data in enumerate(testloader, 0):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        outputs, kl = net(inputs)

        loss = (criterion(outputs, labels)) + kl

        running_loss += loss.item()
        
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
            
    loss_mat_corr[n_index] = running_loss**2
    
    net.features[l_index].bn1.weight.data = weight_base.clone().detach()
    net.features[l_index].bn1.bias.data = bias_base.clone().detach()

In [ ]:
### normal AlexNet

loss_mat_corr = torch.zeros(weight_base.shape[0])

for n_index in range(weight_base.shape[0]): 
    num_stop = 0
    print(n_index)
    running_loss = 0.0

    net.features[l_index].weight.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
    net.features[l_index].bias.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
    
#     for i, data in enumerate(testloader, 0):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        outputs = net(inputs)

        loss = (criterion(outputs, labels))

        running_loss += loss.item()
        
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
            
    loss_mat_corr[n_index] = running_loss**2
    
    net.features[l_index].weight.data = weight_base.clone().detach()
    net.features[l_index].bias.data = bias_base.clone().detach()

In [ ]:
# torch.save(loss_mat_corr, './w_decorr/loss_mats/corr/'+str(l_index)+'/loss_corr_bn_train_'+str(l_index)+'.pt')
loss_mat_corr = torch.load('./w_decorr/loss_mats/corr/'+str(l_index)+'/loss_corr_bn_train_'+str(l_index)+'.pt')

In [ ]:

### SBP AlexNet
optimizer = optim.SGD(net.parameters(), lr=0, weight_decay=0)
av_corrval = 0
n_epochs = 1

for epoch in tqdm(range(n_epochs)):
    num_stop = 0
    running_loss = 0.0
    imp_corr_conv = torch.zeros(bias_base.shape[0]).to(device)
    imp_corr_bn = torch.zeros(bias_base.shape[0]).to(device)
    
    for i, data in enumerate(trainloader, 0):
#     for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs, kl = net(inputs) + kl
        loss = criterion(outputs, labels)
        loss.backward()
        
        imp_corr_bn += (((net.features[l_index].bn1.weight.grad)*(net.features[l_index].bn1.weight.data)) + ((net.features[l_index].bn1.bias.grad)*(net.features[l_index].bn1.bias.data))).abs().pow(2)
        
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
         
    corrval = (np.corrcoef(imp_corr_bn.cpu().detach().numpy(), (loss_mat_corr - loss_base_corr).abs().cpu().detach().numpy()))
    print("Correlation at epoch "+str(epoch)+": "+str(corrval[0,1]))
    av_corrval += corrval[0,1]

In [ ]:

### Normal AlexNet
optimizer = optim.SGD(net.parameters(), lr=0, weight_decay=0)
av_corrval = 0
n_epochs = 1

for epoch in range(n_epochs):
    num_stop = 0
    running_loss = 0.0
    imp_corr_conv = torch.zeros(bias_base.shape[0]).to(device)
    imp_corr_bn = torch.zeros(bias_base.shape[0]).to(device)
    
    for i, data in enumerate(trainloader, 0):
#     for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        imp_corr_bn += (((net.features[l_index].weight.grad)*(net.features[l_index].weight.data)) + ((net.features[l_index].bias.grad)*(net.features[l_index].bias.data))).abs().pow(2)
        
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
         
    corrval = (np.corrcoef(imp_corr_bn.cpu().detach().numpy(), (loss_mat_corr - loss_base_corr).abs().cpu().detach().numpy()))
    print("Correlation at epoch "+str(epoch)+": "+str(corrval[0,1]))
    av_corrval += corrval[0,1]

### Decorrelated net

In [ ]:
net_decorr = AlexNet(cfg).to(device)
net_dict = torch.load('./ortho_checkpoint/ckpt.pth')
net_decorr.load_state_dict(net_dict['net'])
net_decorr = net_decorr.eval() 

In [ ]:
weight_base = net_decorr.features[l_index].weight.data.clone().detach()
bias_base = net_decorr.features[l_index].bias.data.clone().detach()

In [ ]:
optimizer = optim.SGD(net_decorr.parameters(), lr=0, weight_decay=0)
num_stop = 0
loss_base_decorr = 0
for epoch in range(1):
    for i, data in enumerate(trainloader, 0):
#     for i, data in enumerate(testloader, 0):        
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = net_decorr(inputs)
        loss = criterion(outputs, labels)
        loss_base_decorr += loss.item()
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
loss_base_decorr = loss_base_decorr**2

In [ ]:
optimizer = optim.SGD(net_decorr.parameters(), lr=0, weight_decay=0)

loss_mat_decorr = torch.zeros(weight_base.shape[0])

for n_index in range(weight_base.shape[0]): 
    print(n_index)
    num_stop = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
#     for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        net_decorr.features[l_index].weight.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
        net_decorr.features[l_index].bias.data[n_index] = 0 #torch.zeros((weight_base.shape[1],weight_base.shape[2],weight_base.shape[3]))
        outputs = net_decorr(inputs)
        
        loss = criterion(outputs, labels)
        
        running_loss += loss.item()
        
        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
            
    loss_mat_decorr[n_index] = running_loss**2
    
    net_decorr.features[l_index].weight.data = weight_base.clone().detach()
    net_decorr.features[l_index].bias.data = bias_base.clone().detach()

In [ ]:
# torch.save(loss_mat_decorr, './w_decorr/loss_mats/decorr/'+str(l_index)+'/loss_decorr_bn_train_'+str(l_index)+'.pt')
loss_mat_decorr = torch.load('./w_decorr/loss_mats/decorr/'+str(l_index)+'/loss_decorr_bn_train_'+str(l_index)+'.pt')

In [ ]:
optimizer = optim.SGD(net_decorr.parameters(), lr=0, weight_decay=0)
av_corrval = 0
n_epochs = 1

for epoch in range(n_epochs):
    num_stop = 0
    imp_decorr_conv = torch.zeros(bias_base.shape[0]).to(device)
    imp_decorr_bn = torch.zeros(bias_base.shape[0]).to(device)

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
#     for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net_decorr(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        num_stop += labels.shape[0]
        if(num_stop > 5000):
            break
        
        imp_decorr_bn += (((net_decorr.features[l_index].weight.grad)*(net_decorr.features[l_index].weight.data)) + ((net_decorr.features[l_index].bias.grad)*(net_decorr.features[l_index].bias.data))).abs().pow(2)
    
    corrval = (np.corrcoef(imp_decorr_bn.cpu().detach().numpy(), (loss_mat_decorr - loss_base_decorr).abs().cpu().detach().numpy()))
    print("Correlation at epoch "+str(epoch)+": "+str(corrval[0,1]))
    av_corrval += corrval[0,1]

# Graphs

In [ ]:
figure(figsize=(20,5))
s = imp_corr_bn.cpu().sort()[0].cpu().numpy()
order = imp_corr_bn.sort()[1].cpu().numpy()
plt.plot(s/s.max(), label="Estimated importance")
plt.title("Correlated (Taylor FO) for "+str(l_index))
loss_diff = (loss_mat_decorr - loss_base_decorr).abs()
plt.xlabel("Neuron index")
plt.ylabel("Normalized importance")
plt.plot(loss_diff[order]/loss_diff.max(), label="Actual importance")
plt.legend()
plt.savefig("./w_decorr/loss_mats/corr/graphs/"+str(l_index)+".png")

In [ ]:
figure(figsize=(20,5))
s = imp_decorr_bn.cpu().sort()[0].cpu().numpy()
order = imp_decorr_bn.sort()[1].cpu().numpy()
plt.plot(s/s.max(), label="Estimated importance")
plt.title("Decorrelated (Taylor FO) for "+str(l_index))
loss_diff = (loss_mat_decorr - loss_base_decorr).abs()
plt.xlabel("Neuron index")
plt.ylabel("Normalized importance")
plt.plot(loss_diff[order]/loss_diff.max(), label="Actual importance")
plt.legend()
plt.savefig("./w_decorr/loss_mats/decorr/graphs/"+str(l_index)+".png")

In [ ]:
s = imp_decorr_bn.cpu().sort()[0].cpu().numpy()
s = s/s.max()
order = imp_decorr_bn.sort()[1].cpu().numpy()
loss_diff = (loss_mat_decorr - loss_base_decorr).abs()
loss_diff = (loss_diff[order]/loss_diff.max())
ortho_rms = ((loss_diff - s)**2).sum()

s = imp_corr_bn.cpu().sort()[0].cpu().numpy()
s = s/s.max()
order = imp_corr_bn.sort()[1].cpu().numpy()
loss_diff = (loss_mat_corr - loss_base_corr).abs()
loss_diff = (loss_diff[order]/loss_diff.max())

base_rms = ((loss_diff - s)**2).sum()

In [ ]:
(ortho_rms, base_rms)

In [ ]:
# rms_ortho = np.sqrt(np.array(rms_ortho) / np.array([64, 64, 128, 128, 256, 256, 512, 512, 512, 512]))
# rms_base = np.sqrt(np.array(rms_base) / np.array([64, 64, 128, 128, 256, 256, 512, 512, 512, 512]))

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(np.linspace(0,30,10)-0.5, rms_ortho, label="Decorrelated network")
plt.bar(np.linspace(0,30,10)+0.5, rms_base, label="Correlated network")
plt.xlabel("Layer ID")
plt.ylabel("RMS")
plt.legend()
plt.savefig("./w_decorr/loss_mats/rms.png")

## Subplots

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,5))

s = imp_decorr_bn.cpu().sort()[0].cpu().numpy()
order = imp_decorr_bn.sort()[1].cpu().numpy()
axes[0].plot(s/s.max(), label="Estimated importance")
axes[0].set_title("Decorrelated Network (layer "+str(l_index)+")")
loss_diff = (loss_mat_decorr - loss_base_decorr).abs()
axes[0].set_xlabel("Neuron index")
axes[0].set_ylabel("Normalized importance")
axes[0].plot(loss_diff[order]/loss_diff.max(), label="Actual importance")
axes[0].legend()

s = imp_corr_bn.cpu().sort()[0].cpu().numpy()
order = imp_corr_bn.sort()[1].cpu().numpy()
axes[1].plot(s/s.max(), label="Estimated importance")
axes[1].set_title("Correlated Network (layer "+str(l_index)+")")
loss_diff = (loss_mat_corr - loss_base_corr).abs()
axes[1].set_xlabel("Neuron index")
axes[1].set_ylabel("Normalized importance")
axes[1].plot(loss_diff[order]/loss_diff.max(), label="Actual importance")
axes[1].legend()

plt.savefig("./w_decorr/loss_mats/subplots/"+str(l_index)+".png")

# Other metrics

### Net-Slim Train

In [ ]:
scale_corr = net.features[l_index].weight.data.clone()
np.corrcoef(scale_corr.cpu().numpy(), (loss_mat_corr - loss_base_corr).abs().cpu().numpy())

In [ ]:
scale_decorr = net_decorr.features[l_index].weight.data.clone().abs()
np.corrcoef((scale_decorr).cpu().numpy(), (loss_mat_decorr - loss_base_decorr).abs().cpu().numpy())

### L2 based pruning Train

In [ ]:
w_corr = net.features[l_index - 2].weight.data.clone()
w_imp_corr = w_corr.pow(2).sum(dim=(1,2,3)).cpu()
np.corrcoef(w_imp_corr.numpy(), (loss_mat_corr - loss_base_corr).abs().cpu().numpy())

In [ ]:
w_decorr = net_decorr.features[l_index - 2].weight.data.clone()
w_imp_decorr = w_decorr.pow(2).sum(dim=(1,2,3)).cpu()
w_imp_decorr = (w_imp_decorr - w_imp_decorr.min())
w_imp_decorr = w_imp_decorr/w_imp_decorr.max()
np.corrcoef(w_imp_decorr.numpy(), (loss_mat_decorr - loss_base_decorr).abs().cpu().numpy())

#### Importance plots Netslim Train

In [ ]:
figure(figsize=(20,5))

s = scale_corr.cpu().sort()[0].cpu().numpy()
order = scale_corr.sort()[1].cpu().numpy()
plt.plot(s/s.max())
plt.title("Correlated (Net-Slim)")
loss_diff = (loss_mat_corr - loss_base_corr).abs()
plt.plot(loss_diff[order]/loss_diff.max())

In [ ]:
figure(figsize=(20,5))

s = scale_decorr.cpu().sort()[0].cpu().numpy()
order = scale_decorr.sort()[1].cpu().numpy()
plt.plot(s/s.max())
plt.title("Decorrelated (Net-Slim)")
loss_diff = (loss_mat_decorr - loss_base_decorr).abs()
plt.plot(loss_diff[order]/loss_diff.max())

#### Importance plots L2 train

In [ ]:
figure(figsize=(20,5))
s = w_imp_corr.sort()[0].cpu().numpy()
order = w_imp_corr.sort()[1].cpu().numpy()
plt.plot(s/s.max())
plt.title("Correlated (L2)")
loss_diff = (loss_mat_corr - loss_base_corr).abs()
plt.plot(loss_diff[order]/loss_diff.max())

In [ ]:
figure(figsize=(20,5))
s = w_imp_decorr.sort()[0].cpu().numpy()
order = w_imp_decorr.sort()[1].cpu().numpy()
plt.plot(s/s.max())
plt.title("Decorrelated (L2)")
loss_diff = (loss_mat_decorr - loss_base_decorr).abs()
plt.plot(loss_diff[order]/loss_diff.max())